In [15]:
import os
import pandas as pd
import csv

def detect_delimiter(file_path):
    with open(file_path, 'r', newline='', encoding='utf-8') as file:
        dialect = csv.Sniffer().sniff(file.read(1024))
    return dialect.delimiter

delimiter = detect_delimiter('ERIHPLUSapprovedJournals.csv')
erih_plus_df = pd.read_csv('ERIHPLUSapprovedJournals.csv', sep=delimiter)

In [16]:
erih_plus_df.head(1)

,Journal ID,Print ISSN,Online ISSN,Original Title,International Title,Country of Publication,ERIH PLUS Disciplines,OECD Classifications,[Last Updated]
0,486254,1989-3477,NaN,@tic.revista d'innovació educativa,@tic.revista d'innovació educativa,Spain,Interdisciplinary research in the Social Scien...,Educational Sciences; Other Social Sciences,2015-06-25 13:48:26


In [17]:
def process_meta_csv(file_path, erih_plus_df):
    meta_data = pd.read_csv(file_path)
    meta_data['venue'] = meta_data['venue'].astype(str)
    meta_data['issn'] = meta_data['venue'].str.extract(r'issn:(\d{4}-\d{3}[\dX])')
    
    # Extract the identifier (OMID) from the 'id' column
    meta_data['id'] = meta_data['id'].str.extract(r'(meta:[^ ]*)')
    
    merged_data_print = erih_plus_df.merge(meta_data, left_on='Print ISSN', right_on='issn', how='inner')
    merged_data_online = erih_plus_df.merge(meta_data, left_on='Online ISSN', right_on='issn', how='inner')
    merged_data = pd.concat([merged_data_print, merged_data_online], ignore_index=True)
    
    # Keep only the relevant columns for the mapping dataframe
    merged_data = merged_data[['id', 'issn', 'Journal ID', 'Print ISSN', 'Online ISSN']].rename(columns={'id': 'OC_OMID', 'issn': 'OC_ISSN', 'Journal ID': 'EP_ID', 'Print ISSN': 'EP_Print_ISSN', 'Online ISSN': 'EP_Online_ISSN'})
    
    # Create the 'EP_ISSN' column
    merged_data['EP_ISSN'] = merged_data['EP_Print_ISSN'].combine_first(merged_data['EP_Online_ISSN'])
    
    # Drop the 'EP_Print_ISSN' and 'EP_Online_ISSN' columns
    merged_data = merged_data.drop(columns=['EP_Print_ISSN', 'EP_Online_ISSN'])

    return merged_data



In [18]:
csv_directory = 'I:\\open-sci\\dump-files\\opencitations-meta\\solo_one'
merged_data = pd.DataFrame()

for file_name in os.listdir(csv_directory):
    if file_name.endswith('.csv'):
        file_path = os.path.join(csv_directory, file_name)
        merged_data_file = process_meta_csv(file_path, erih_plus_df)
        merged_data = pd.concat([merged_data, merged_data_file], ignore_index=True)

In [5]:
#merged_data.to_csv('OpenCitations_Meta_ERIH_PLUS_mapping_3.csv', index=False)

In [19]:
merged_data.head(1)

,OC_OMID,OC_ISSN,EP_ID,EP_ISSN
0,meta:br/060100,NaN,488561,2341-0515


In [20]:
new_merged_data = merged_data.dropna(subset=['OC_ISSN']).reset_index(drop=True)
new_merged_data.head(2)

,OC_OMID,OC_ISSN,EP_ID,EP_ISSN
0,meta:br/0601646,0172-6404,471777,0172-6404
1,meta:br/0601638,0172-6404,471777,0172-6404
